In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import shutil
import cv2
import os
from tqdm import tqdm
%pip install ultralytics
import ultralytics
ultralytics.checks()
from sklearn.model_selection import train_test_split

In [ ]:
# Concatenate data paths with labels into one dataframe ( to later be fitted into the model )
def create_df(data_dir):
    csv_paths = glob.glob(f'{data_dir}/*.csv')
    l=[]
    for i in csv_paths:
        df=pd.read_csv(i, index_col=None,header=0)
        l.append(df)
    return pd.concat(l)

def split_data(data_dir):
    df = create_df(data_dir)
    train_df, dummy_df = train_test_split(df, train_size= 0.8, shuffle= True, random_state= 123, stratify= df["class"])

    # valid and test dataframe
    valid_df, test_df = train_test_split(dummy_df,  train_size= 0.5, shuffle= True, random_state= 123, stratify= dummy_df["class"])

    return train_df, valid_df, test_df, sorted(df["class"].unique())

def generate_data(df, classes, data_path, dest_path):
    os.makedirs(f'{dest_path}/images', exist_ok=True)
    os.makedirs(f'{dest_path}/labels', exist_ok=True)
    classes=np.array(classes)
    for i, row in df.iterrows():
        jpg_file_path = f'{dest_path}/images/{row["filename"]}.jpg'
        txt_file_path = f'{dest_path}/labels/{row["filename"]}.txt'
        shutil.copy(f'{data_path}/{row["filename"]}.jpg', jpg_file_path)
        with open(txt_file_path, mode='a') as f:
            try:
                class_num = np.where(classes==row["class"])[0][0]
                width=row["width"]
                height=row["height"]
                xmin = row["xmin"]
                ymin = row["ymin"]
                xmax = row["xmax"]
                ymax = row["ymax"]
                x_center = 0.5*(xmin+xmax)
                y_center = 0.5*(ymin+ymax)
                b_width = xmax - xmin
                b_height= ymax - ymin
                output_string = '{} {} {} {} {}\n'.format(class_num,
                                                        x_center/width,
                                                        y_center/height,
                                                        b_width/width,
                                                        b_height/height)
                f.write(output_string)
            except:
                print(txt_file_path)
                0/0
                

data_path="/kaggle/input/militaryaircraftdetectiondataset/dataset"
# split data into train, val and test
working_path="/kaggle/working"
tr_df, val_df, ts_df, classes=split_data(data_path)
train_path=f"{working_path}/data/train"
valid_path=f"{working_path}/data/valid"
test_path=f"{working_path}/data/test"
generate_data(tr_df,classes,data_path,train_path)
generate_data(val_df,classes,data_path,test_path)
generate_data(ts_df,classes,data_path,valid_path)

# generate train and test yaml
classes_string = ', '.join([f'"{c}"' for c in classes])
# training
with open(f'{working_path}/train.yaml', mode='w') as f:
    yaml_string=f'train: {train_path}\n'\
              + f'val: {valid_path}\n'\
              + f'nc: {len(classes)}\n'\
              + f'names: [{classes_string}]'
    f.write(yaml_string)
# testing
with open(f'{working_path}/test.yaml', mode='w') as f:
    yaml_string=f'train: {train_path}\n'\
              + f'val: {test_path}\n'\
              + f'nc: {len(classes)}\n'\
              + f'names: [{classes_string}]'
    f.write(yaml_string)

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8m.pt')
# train the model
result=model.train(
    name='YOLOv8m_1',
    device=0,
    epochs=10,
    imgsz=640,
    batch=32,
    lr0=1e-2,
    data=f'{working_path}/train.yaml',
)

In [ ]:
# test the model
!yolo val \
model='/kaggle/input/amlcw/runs/detect/YOLOv8m_1/weights/best.pt' \
data='/kaggle/working/test.yaml'\
batch=4 \
device=cuda:0 \
imgsz=640

for path in sorted(glob.glob('/kaggle/working/runs/detect/val/*.png')):
    image = cv2.imread(path)[:,:,::-1]
    plt.figure(figsize=(20,20))
    plt.imshow(image)
    plt.show()
    
for path in sorted(glob.glob('/kaggle/working/runs/detect/val/*.jpg')):
    image = cv2.imread(path)[:,:,::-1]
    plt.figure(figsize=(20,20))
    plt.imshow(image)
    plt.show()